In [25]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'project-eda:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4515548%2F7728378%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240302%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240302T111000Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D31ad307cc6e4195011e982ecf5cee2b4eef6d06997d9d3fef6359cb65bbaf5c4c63bd220a496b720d5e233cb2d8d75058e274ab435bd89a220bc87327088b8d89935c1ecb16bbfddd88ceae450dedd63668eee36036b6da25b83bd0f9cd81090761cb57d35ab6e75c246e8e6ccda56bcf06bfcdf9bbc20dc8b88de63b3d2c3b0c0132acb02d8073df35bd288442593b51e18dc3f111e388a417d527f61627da31d3e22451bfe8e7c779bcd61611f38f8caced82c34a43f9a1058e24e7189b330548839c756a9ca4587ec2536bcaf939752fec6938f9bf128cf7260f6ed8bf20ca846124104cbcdf4aaa47c54f5a937374c45ce2a998a28101b3ba81e3d63809c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 304720100 bytes downloaded
Downloaded and uncompressed: project-eda
Data source import complete.


In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/project-eda/category_tree.csv
/kaggle/input/project-eda/events.csv/events.csv
/kaggle/input/project-eda/item_properties_part2.csv/item_properties_part2.csv
/kaggle/input/project-eda/item_properties_part1.csv/item_properties_part1.csv


Загружаем датасеты

In [27]:
df_events = pd.read_csv('/kaggle/input/project-eda/events.csv/events.csv')
df_category_tree = pd.read_csv('/kaggle/input/project-eda/category_tree.csv')
df_properties_part1 = pd.read_csv('/kaggle/input/project-eda/item_properties_part1.csv/item_properties_part1.csv')
df_properties_part2 = pd.read_csv('/kaggle/input/project-eda/item_properties_part2.csv/item_properties_part2.csv')

Посмотрим на основной датасет

In [28]:
df_events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


Переведем признак timestamp в нормальный вид

In [29]:
df_events['timestamp'] = pd.to_datetime(df_events['timestamp'], unit='ms', origin='unix')
df_events

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 05:02:12.117,257597,view,355908,NaN
1,2015-06-02 05:50:14.164,992329,view,248676,NaN
2,2015-06-02 05:13:19.827,111016,view,318965,NaN
3,2015-06-02 05:12:35.914,483717,view,253185,NaN
4,2015-06-02 05:02:17.106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,2015-08-01 03:13:05.939,591435,view,261427,NaN
2756097,2015-08-01 03:30:13.142,762376,view,115946,NaN
2756098,2015-08-01 02:57:00.527,1251746,view,78144,NaN
2756099,2015-08-01 03:08:50.703,1184451,view,283392,NaN


Объединим датасеты df_properties_part1 и df_properties_part2

In [30]:
frames = [df_properties_part1, df_properties_part2]
df_properties_part = pd.concat(frames)

Неделя 1

Задание 2.1

Сколько записей событий находится в датасете? Ответ введите без пробелов и знаков препинания.

In [31]:
len(df_events)

2756101

Задание 2.2

Какие типы событий содержатся в датасете? Выберите все верные варианты.

In [32]:
df_events['event'].unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

Задание 2.3

Сколько уникальных признаков товара есть в датасете? Ответ вводите без пробелов и знаков препинания.

In [33]:
df_properties_part['property'].nunique()

1104

Задание 4.1

Какой процент продаж обеспечивают топовые товары (точка отсечения — 1 июля)? Для расчёта используйте топ-3 товаров по числу транзакций.

Ответ введите без знака процента.

Топ- 3

In [34]:
df_events_07 = df_events[df_events['timestamp']<='2015-07-01']
df_top3 = df_events_07.groupby('itemid').agg({'transactionid': 'max'}).sort_values(by='transactionid', ascending=False).iloc[:3]
df_top3

,transactionid
itemid,
5470,17671.0
461686,17670.0
132418,17669.0


In [35]:
df_events_before = df_events[(df_events['timestamp']<='2015-07-01') &(df_events['event']=='transaction')]
df_events_before.groupby('itemid')['transactionid'].count().sort_values(ascending=False).sum()

9654

In [36]:
top_before = df_events_before.groupby('itemid')['transactionid'].count().sort_values(ascending=False)[:3].index.to_list()
top_before

[119736, 369447, 7943]

In [37]:
df_events_after = df_events[(df_events['timestamp']>='2015-07-02') &(df_events['event']=='transaction')]
df_events_after_sum = df_events_after[df_events_after['itemid'].isin(top_before )].groupby('itemid')['transactionid'].count().sum()
df_events_after, df_events_after_sum

(                      timestamp  visitorid        event  itemid  transactionid
 610656  2015-08-02 05:35:44.445    1318304  transaction   72721         5851.0
 611372  2015-08-01 23:42:02.831    1293358  transaction  340794        12729.0
 611382  2015-08-01 23:11:34.133     838490  transaction  235047         7283.0
 611465  2015-08-01 17:07:35.547    1229936  transaction  226089        16793.0
 611804  2015-08-01 20:09:52.635     705542  transaction  119298         7951.0
 ...                         ...        ...          ...     ...            ...
 2755294 2015-07-31 21:12:56.570    1050575  transaction   31640         8354.0
 2755349 2015-07-31 21:57:58.779     861299  transaction  456602         3643.0
 2755508 2015-07-31 15:48:50.123     855941  transaction  235771         4385.0
 2755603 2015-07-31 15:12:40.300     548772  transaction   29167        13872.0
 2755607 2015-07-31 16:09:49.163    1051054  transaction  312728        17579.0
 
 [12608 rows x 5 columns],
 77)

In [38]:
df_events_after_gr = df_events_after.groupby('itemid')['transactionid'].count().sum()
df_events_after_gr

12608

In [39]:
PR = df_events_after_sum / df_events_after_gr *100
print('Процент продаж товаров топ-3:', round(PR, 2))

Процент продаж товаров топ-3: 0.61


Работаем дальше

In [40]:
df_category_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669 entries, 0 to 1668
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   categoryid  1669 non-null   int64  
 1   parentid    1644 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 26.2 KB


In [41]:
df_category_tree['categoryid'].unique ()

array([1016,  809,  570, ..., 1336,  689,  761])

In [42]:
df_category_tree.nunique ()

categoryid    1669
parentid       362
dtype: int64

In [43]:
df_category_tree

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0
...,...,...
1664,49,1125.0
1665,1112,630.0
1666,1336,745.0
1667,689,207.0


In [44]:
df_properties_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20275902 entries, 0 to 9275902
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   itemid     int64 
 2   property   object
 3   value      object
dtypes: int64(2), object(2)
memory usage: 773.5+ MB


In [45]:
df_properties_part.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [46]:
df_properties_part.nunique ()

timestamp         18
itemid        417053
property        1104
value        1966868
dtype: int64

неделя 2

In [47]:
train, test = train_test_split(df_events, test_size=0.3, shuffle=False)
train, test

(                      timestamp  visitorid event  itemid  transactionid
 0       2015-06-02 05:02:12.117     257597  view  355908            NaN
 1       2015-06-02 05:50:14.164     992329  view  248676            NaN
 2       2015-06-02 05:13:19.827     111016  view  318965            NaN
 3       2015-06-02 05:12:35.914     483717  view  253185            NaN
 4       2015-06-02 05:02:17.106     951259  view  367447            NaN
 ...                         ...        ...   ...     ...            ...
 1929265 2015-05-25 00:17:22.621     253882  view  310746            NaN
 1929266 2015-05-24 23:55:28.675     294666  view  250851            NaN
 1929267 2015-05-25 00:25:39.884    1063108  view  188480            NaN
 1929268 2015-05-25 00:22:23.190    1127743  view  159041            NaN
 1929269 2015-05-25 00:10:52.691     279917  view    4909            NaN
 
 [1929270 rows x 5 columns],
                       timestamp  visitorid event  itemid  transactionid
 1929270 2015-05-24 